In [1]:
import pandas as pd
import os
import pathlib

# 获取项目根目录

In [2]:
root = pathlib.Path(os.path.abspath('__file__')).parent.parent
root

WindowsPath('D:/Learning/Project/QA/version1')

#  gensim实践：



使用Gensim这个包就可以进行我们word2vec的完整的训练过程，训练后得到词向量

1. 读取预处理好的数据
2. 训练
3. 完事

# 数据集路径

In [26]:
merger_data_path = os.path.join(root, 'data', 'merged_train_test_seg_data.csv')

# 载入数据

In [27]:
# 这里其实并不需要载入这部分数据，只是载入进来看一下
# 载入的时候设置不读header
merger_df = pd.read_csv(merger_data_path,header=None)
print('merger_data_path data size {}'.format(len(merger_df)))
merger_df.head()

merger_data_path data size 102871


,0
0,方向机 重 助力 泵 方向机 都 换 新 都 换 助力 泵 方向机 换 方向机 带 助力 重...
1,奔驰 ML500 排气 凸轮轴 调节 错误 有没有 电脑 检测 故障 代码 有发 一下 发动...
2,2010 款 宝马X1 2011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...
3,30V6 发动机 号 位置 照片 最好 右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 ...
4,2012 款 奔驰 c180 维修保养 动力 值得 拥有 家庭 用车 入手 维修保养 费用 ...


# 模型创建

Gensim中 Word2Vec 模型的期望输入是进过分词的句子列表，即是某个二维数组。这里我们暂时使用 Python 内置的数组，不过其在输入数据集较大的情况下会占用大量的 RAM。Gensim 本身只是要求能够迭代的有序句子列表，因此在工程实践中我们可以使用自定义的生成器，只在内存中保存单条语句。

## Word2Vec 参数
+ min_count

在不同大小的语料集中，我们对于基准词频的需求也是不一样的。譬如在较大的语料集中，我们希望忽略那些只出现过一两次的单词，这里我们就可以通过设置min_count参数进行控制。一般而言，合理的参数值会设置在0~100之间。

+ size

<span class="burk">size参数主要是用来设置神经网络的层数，即词向量的大小</span>。Word2Vec 中的默认值是设置为100层。更大的层次设置意味着更多的输入数据，不过也能提升整体的准确度，合理的设置范围为 10~数百。

+ workers

workers参数用于设置并发训练时候的线程数，不过仅当Cython安装的情况下才会起作用：
最好定义成机器的核数。

In [14]:
# 引入 word2vec
from gensim.models.word2vec import LineSentence  # 可以按行读取训练好的语料，传入语料的路径自己就会去读
from gensim.models import word2vec
import gensim

# 引入日志配置
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 构建训练

In [15]:
merger_data_path

'D:/Learning/Project/QA/data/merged_train_test_seg_data.csv'

In [16]:
model = word2vec.Word2Vec(LineSentence(merger_data_path), workers=8,min_count=5,size=200)
"""
这里传入训练数据的路径，自己会去读，每一行就是一个字符串，通过LineSentencee就会得到sentence,也就是
每一行的数据。然后定义用几个workers来执行，min_count就是之前讲的词频，这里词频小于5的就把它滤掉，这里
的size就是词向量训练的维度
"""


2020-05-16 18:21:46,724 : INFO : collecting all words and their counts
2020-05-16 18:21:46,725 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-16 18:21:46,956 : INFO : PROGRESS: at sentence #10000, processed 937272 words, keeping 36653 word types
2020-05-16 18:21:47,217 : INFO : PROGRESS: at sentence #20000, processed 1889029 words, keeping 53934 word types
2020-05-16 18:21:47,478 : INFO : PROGRESS: at sentence #30000, processed 2829436 words, keeping 66707 word types
2020-05-16 18:21:47,728 : INFO : PROGRESS: at sentence #40000, processed 3741910 words, keeping 77608 word types
2020-05-16 18:21:47,992 : INFO : PROGRESS: at sentence #50000, processed 4714600 words, keeping 87459 word types
2020-05-16 18:21:48,278 : INFO : PROGRESS: at sentence #60000, processed 5748570 words, keeping 97386 word types
2020-05-16 18:21:48,559 : INFO : PROGRESS: at sentence #70000, processed 6805872 words, keeping 106961 word types
2020-05-16 18:21:48,788 : INFO : PROGRE

2020-05-16 18:22:19,346 : INFO : EPOCH - 4 : training on 9704885 raw words (8594576 effective words) took 6.2s, 1385191 effective words/s
2020-05-16 18:22:20,354 : INFO : EPOCH 5 - PROGRESS: at 14.70% examples, 1251419 words/s, in_qsize 13, out_qsize 2
2020-05-16 18:22:21,367 : INFO : EPOCH 5 - PROGRESS: at 30.79% examples, 1309702 words/s, in_qsize 6, out_qsize 0
2020-05-16 18:22:22,372 : INFO : EPOCH 5 - PROGRESS: at 46.50% examples, 1316385 words/s, in_qsize 16, out_qsize 0
2020-05-16 18:22:23,384 : INFO : EPOCH 5 - PROGRESS: at 62.22% examples, 1353427 words/s, in_qsize 15, out_qsize 0
2020-05-16 18:22:24,388 : INFO : EPOCH 5 - PROGRESS: at 77.32% examples, 1353312 words/s, in_qsize 8, out_qsize 3
2020-05-16 18:22:25,388 : INFO : EPOCH 5 - PROGRESS: at 95.92% examples, 1370149 words/s, in_qsize 14, out_qsize 0
2020-05-16 18:22:25,564 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-05-16 18:22:25,571 : INFO : worker thread finished; awaiting finish of 6 more 

'\n这里传入训练数据的路径，自己会去读，每一行就是一个字符串，通过LineSentencee就会得到sentence,也就是\n每一行的数据。然后定义用几个workers来执行，min_count就是之前讲的词频，这里词频小于5的就把它滤掉，这里\n的size就是词向量训练的维度\n'

# 查找最近的词

```
这里是一个建立词向量的过程，得到的是词向量的结果，是对应着每一个词的词向量，所以可以进行一些词级别的操作，如计算词与词之间的相似度等
```

```
这里下边拿到训练好的word2vec之后赋给一个model，model里边的wv里边有一个相似度计算的包，
放进去一个奇瑞，可以直接计算一下相似度。这里的参数topn表示相似度最高的多少个词
```

In [17]:
model.wv.most_similar(['奇瑞'], topn=10)


2020-05-16 18:22:25,609 : INFO : precomputing L2-norms of word weight vectors


[('东南', 0.8473502397537231),
 ('瑞虎5', 0.8420183658599854),
 ('名爵', 0.8392105102539062),
 ('江淮', 0.8293333649635315),
 ('福美来', 0.8245681524276733),
 ('二代', 0.8205920457839966),
 ('铃木', 0.8200820684432983),
 ('s3', 0.819768488407135),
 ('海马', 0.8177907466888428),
 ('鹰', 0.8176668882369995)]

# 保存模型

In [28]:
save_model_path = os.path.join(root, 'data/wv', 'word2vec.model')

In [29]:
model.save(save_model_path)

2020-05-16 18:31:44,249 : INFO : saving Word2Vec object under D:\Learning\Project\QA\data/wv\word2vec.model, separately None
2020-05-16 18:31:44,249 : INFO : not storing attribute vectors_norm
2020-05-16 18:31:44,250 : INFO : not storing attribute cum_table
2020-05-16 18:31:44,706 : INFO : saved D:\Learning\Project\QA\data/wv\word2vec.model


# 载入模型

In [30]:
model = word2vec.Word2Vec.load(save_model_path)

2020-05-16 18:31:47,702 : INFO : loading Word2Vec object from D:\Learning\Project\QA\data/wv\word2vec.model
2020-05-16 18:31:48,237 : INFO : loading wv recursively from D:\Learning\Project\QA\data/wv\word2vec.model.wv.* with mmap=None
2020-05-16 18:31:48,238 : INFO : setting ignored attribute vectors_norm to None
2020-05-16 18:31:48,239 : INFO : loading vocabulary recursively from D:\Learning\Project\QA\data/wv\word2vec.model.vocabulary.* with mmap=None
2020-05-16 18:31:48,240 : INFO : loading trainables recursively from D:\Learning\Project\QA\data/wv\word2vec.model.trainables.* with mmap=None
2020-05-16 18:31:48,241 : INFO : setting ignored attribute cum_table to None
2020-05-16 18:31:48,241 : INFO : loaded D:\Learning\Project\QA\data/wv\word2vec.model


In [31]:
model.wv.most_similar(['奇瑞'],topn=10)

2020-05-16 18:31:48,846 : INFO : precomputing L2-norms of word weight vectors


[('东南', 0.8473502397537231),
 ('瑞虎5', 0.8420183658599854),
 ('名爵', 0.8392105102539062),
 ('江淮', 0.8293333649635315),
 ('福美来', 0.8245681524276733),
 ('二代', 0.8205920457839966),
 ('铃木', 0.8200820684432983),
 ('s3', 0.819768488407135),
 ('海马', 0.8177907466888428),
 ('鹰', 0.8176668882369995)]

# 参考

1. https://radimrehurek.com/gensim/models/word2vec.html 